In [2]:
import pandas as pd
from pathlib import Path


In [3]:
DATA_DIR = Path("../data/raw")

fi_data = pd.read_excel(
    DATA_DIR / "ethiopia_fi_unified_data.xlsx",
    sheet_name=0
)

impact_links = pd.read_excel(
    DATA_DIR / "ethiopia_fi_unified_data.xlsx",
    sheet_name=1
)

reference_codes = pd.read_excel(
    DATA_DIR / "reference_codes.xlsx"
)


In [4]:
fi_data.info()
fi_data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   record_id            43 non-null     object        
 1   record_type          43 non-null     object        
 2   category             10 non-null     object        
 3   pillar               33 non-null     object        
 4   indicator            43 non-null     object        
 5   indicator_code       43 non-null     object        
 6   indicator_direction  33 non-null     object        
 7   value_numeric        33 non-null     float64       
 8   value_text           10 non-null     object        
 9   value_type           43 non-null     object        
 10  unit                 33 non-null     object        
 11  observation_date     43 non-null     datetime64[ns]
 12  period_start         10 non-null     datetime64[ns]
 13  period_end           10 non-null     

,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,REC_0001,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,22.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Baseline year,NaN
1,REC_0002,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,35.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
2,REC_0003,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,46.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
3,REC_0004,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,56.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN
4,REC_0005,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,36.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN


In [20]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("../data/raw")

reference_codes = pd.read_excel(DATA_DIR / "reference_codes.xlsx")

# Inspect columns
print(reference_codes.columns.tolist())
reference_codes.head()

# Validate record_type, pillar, indicator_code
valid_record_types = reference_codes[reference_codes['field'] == 'record_type']['code']
valid_pillars = reference_codes[reference_codes['field'] == 'pillar']['code']
valid_indicators = reference_codes[reference_codes['field'] == 'indicator_code']['code']

# Check dataset for invalid entries
fi_data = pd.read_excel(DATA_DIR / "ethiopia_fi_unified_data.xlsx", sheet_name=0)

invalid_record_types = fi_data[~fi_data['record_type'].isin(valid_record_types)]
invalid_pillars = fi_data[fi_data['pillar'].notna() & ~fi_data['pillar'].isin(valid_pillars)]
invalid_indicators = fi_data[fi_data['indicator_code'].notna() & ~fi_data['indicator_code'].isin(valid_indicators)]

print("Invalid record_type:", invalid_record_types)
print("Invalid pillars:", invalid_pillars)
print("Invalid indicators:", invalid_indicators)


['field', 'code', 'description', 'applies_to']
Invalid record_type: Empty DataFrame
Columns: [record_id, record_type, category, pillar, indicator, indicator_code, indicator_direction, value_numeric, value_text, value_type, unit, observation_date, period_start, period_end, fiscal_year, gender, location, region, source_name, source_type, source_url, confidence, related_indicator, relationship_type, impact_direction, impact_magnitude, impact_estimate, lag_months, evidence_basis, comparable_country, collected_by, collection_date, original_text, notes]
Index: []

[0 rows x 34 columns]
Invalid pillars: Empty DataFrame
Columns: [record_id, record_type, category, pillar, indicator, indicator_code, indicator_direction, value_numeric, value_text, value_type, unit, observation_date, period_start, period_end, fiscal_year, gender, location, region, source_name, source_type, source_url, confidence, related_indicator, relationship_type, impact_direction, impact_magnitude, impact_estimate, lag_months,

Events do not have a fixed pillar assignment because one event may affect multiple dimensions:

Telebirr launch → increases USAGE and ACCESS

Fayda rollout → affects ACCESS, TRUST, GENDER

Policy changes → may impact multiple indicators indirectly

Assigning a single pillar to events prematurely biases the modeling. We keep pillars only on impact_links where the indicator is explicitly affected.

In [21]:
impact_links = pd.read_excel(DATA_DIR / "ethiopia_fi_unified_data.xlsx", sheet_name=1)
impact_links.head()

# Example explanation
# One event can have multiple impact_links
# Each link defines:
#   - related_indicator (which metric is affected)
#   - pillar (ACCESS / USAGE)
#   - impact_direction (+/-)
#   - impact_magnitude (low/medium/high)
#   - lag_months (delay before effect occurs)


,record_id,parent_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,IMP_0001,EVT_0001,impact_link,NaN,ACCESS,Telebirr effect on Account Ownership,NaN,NaN,15.0,NaN,...,increase,high,15.0,12,literature,Kenya,Example_Trainee,2025-01-20,NaN,Kenya M-Pesa showed +20pp over 5 years
1,IMP_0002,EVT_0001,impact_link,NaN,USAGE,Telebirr effect on Telebirr Users,NaN,NaN,NaN,NaN,...,increase,high,NaN,3,empirical,NaN,Example_Trainee,2025-01-20,NaN,Direct subscriber acquisition
2,IMP_0003,EVT_0001,impact_link,NaN,USAGE,Telebirr effect on P2P Transactions,NaN,NaN,25.0,NaN,...,increase,high,25.0,6,empirical,NaN,Example_Trainee,2025-01-20,NaN,New digital payment channel
3,IMP_0004,EVT_0002,impact_link,NaN,ACCESS,Safaricom effect on 4G Coverage,NaN,NaN,15.0,NaN,...,increase,medium,15.0,12,empirical,NaN,Example_Trainee,2025-01-20,NaN,Network investment from competition
4,IMP_0005,EVT_0002,impact_link,NaN,AFFORDABILITY,Safaricom effect on Data Affordability,NaN,NaN,-20.0,NaN,...,decrease,medium,-20.0,12,literature,Rwanda,Example_Trainee,2025-01-20,NaN,Competition typically reduces prices


All records share the same columns

Interpretation depends on record_type

Many columns will be empty depending on type

In [5]:
fi_data['record_type'].value_counts()


record_type
observation    30
event          10
target          3
Name: count, dtype: int64

observation → measured values

event → something happened

target → official goal

In [6]:
fi_data['indicator_code'].dropna().unique()


array(['ACC_OWNERSHIP', 'ACC_MM_ACCOUNT', 'ACC_4G_COV', 'ACC_MOBILE_PEN',
       'ACC_FAYDA', 'USG_P2P_COUNT', 'USG_P2P_VALUE', 'USG_ATM_COUNT',
       'USG_ATM_VALUE', 'USG_CROSSOVER', 'USG_TELEBIRR_USERS',
       'USG_TELEBIRR_VALUE', 'USG_MPESA_USERS', 'USG_MPESA_ACTIVE',
       'USG_ACTIVE_RATE', 'AFF_DATA_INCOME', 'GEN_GAP_ACC',
       'GEN_MM_SHARE', 'GEN_GAP_MOBILE', 'EVT_TELEBIRR', 'EVT_SAFARICOM',
       'EVT_MPESA', 'EVT_FAYDA', 'EVT_FX_REFORM', 'EVT_CROSSOVER',
       'EVT_MPESA_INTEROP', 'EVT_ETHIOPAY', 'EVT_NFIS2',
       'EVT_SAFCOM_PRICE'], dtype=object)

In [11]:
valid_record_types = reference_codes[
    reference_codes['field'] == 'record_type'
]['code']

fi_data.loc[
    ~fi_data['record_type'].isin(valid_record_types),
    'record_type'
]


Series([], Name: record_type, dtype: object)

In [9]:
reference_codes.head()


,field,code,description,applies_to
0,record_type,observation,Actual measured value from a source,All
1,record_type,event,Policy launch market event or milestone,All
2,record_type,impact_link,Relationship between event and indicator (link...,All
3,record_type,target,Policy target or official goal,All
4,record_type,baseline,Starting point for comparison,All


In [10]:
reference_codes.columns



Index(['field', 'code', 'description', 'applies_to'], dtype='object')

In [12]:
valid_pillars = reference_codes[
    reference_codes['field'] == 'pillar'
]['code']

fi_data.loc[
    fi_data['pillar'].notna() & 
    ~fi_data['pillar'].isin(valid_pillars),
    'pillar'
]


Series([], Name: pillar, dtype: object)

In [13]:
fi_data['pillar'].value_counts(dropna=False)


pillar
ACCESS           16
USAGE            11
NaN              10
GENDER            5
AFFORDABILITY     1
Name: count, dtype: int64

In [14]:
fi_data['source_type'].value_counts(dropna=False)


source_type
operator      15
survey        10
regulator      7
research       4
policy         3
calculated     2
news           2
Name: count, dtype: int64

In [15]:
fi_data['confidence'].value_counts(dropna=False)


confidence
high      40
medium     3
Name: count, dtype: int64

In [16]:
obs = fi_data[fi_data['record_type'] == 'observation']

obs['observation_date'].min(), obs['observation_date'].max()


(Timestamp('2014-12-31 00:00:00'), Timestamp('2025-12-31 00:00:00'))

In [17]:
obs.groupby('indicator_code').size().sort_values(ascending=False)


indicator_code
ACC_OWNERSHIP         6
ACC_FAYDA             3
ACC_4G_COV            2
ACC_MM_ACCOUNT        2
GEN_GAP_ACC           2
USG_P2P_COUNT         2
ACC_MOBILE_PEN        1
GEN_GAP_MOBILE        1
GEN_MM_SHARE          1
USG_ACTIVE_RATE       1
AFF_DATA_INCOME       1
USG_ATM_COUNT         1
USG_ATM_VALUE         1
USG_MPESA_ACTIVE      1
USG_CROSSOVER         1
USG_MPESA_USERS       1
USG_P2P_VALUE         1
USG_TELEBIRR_USERS    1
USG_TELEBIRR_VALUE    1
dtype: int64

In [18]:
events = fi_data[fi_data['record_type'] == 'event']
events[['record_id', 'category', 'indicator', 'observation_date']]


,record_id,category,indicator,observation_date
33,EVT_0001,product_launch,Telebirr Launch,2021-05-17
34,EVT_0002,market_entry,Safaricom Ethiopia Commercial Launch,2022-08-01
35,EVT_0003,product_launch,M-Pesa Ethiopia Launch,2023-08-01
36,EVT_0004,infrastructure,Fayda Digital ID Program Rollout,2024-01-01
37,EVT_0005,policy,Foreign Exchange Liberalization,2024-07-29
38,EVT_0006,milestone,P2P Transaction Count Surpasses ATM,2024-10-01
39,EVT_0007,partnership,M-Pesa EthSwitch Integration,2025-10-27
40,EVT_0008,infrastructure,EthioPay Instant Payment System Launch,2025-12-18
41,EVT_0009,policy,NFIS-II Strategy Launch,2021-09-01
42,EVT_0010,pricing,Safaricom Ethiopia Price Increase,2025-12-15


In [19]:
impact_links[['parent_id', 'pillar', 'related_indicator', 'impact_direction', 'lag_months']]


,parent_id,pillar,related_indicator,impact_direction,lag_months
0,EVT_0001,ACCESS,ACC_OWNERSHIP,increase,12
1,EVT_0001,USAGE,USG_TELEBIRR_USERS,increase,3
2,EVT_0001,USAGE,USG_P2P_COUNT,increase,6
3,EVT_0002,ACCESS,ACC_4G_COV,increase,12
4,EVT_0002,AFFORDABILITY,AFF_DATA_INCOME,decrease,12
5,EVT_0003,USAGE,USG_MPESA_USERS,increase,3
6,EVT_0003,ACCESS,ACC_MM_ACCOUNT,increase,6
7,EVT_0004,ACCESS,ACC_OWNERSHIP,increase,24
8,EVT_0004,GENDER,GEN_GAP_ACC,decrease,24
9,EVT_0005,AFFORDABILITY,AFF_DATA_INCOME,increase,3


In [22]:
import pandas as pd
from pathlib import Path

DATA_RAW = Path("../data/raw")

# Load Sheet 1 (data: observations, events, targets)
fi_data = pd.read_excel(DATA_RAW / "ethiopia_fi_unified_data.xlsx", sheet_name=0)

# Load Sheet 2 (impact_links)
impact_links = pd.read_excel(DATA_RAW / "ethiopia_fi_unified_data.xlsx", sheet_name=1)

# Load reference codes
reference_codes = pd.read_excel(DATA_RAW / "reference_codes.xlsx")


In [23]:
# Print all columns
print("Columns in fi_data:")
print(fi_data.columns.tolist())

# Head of dataset
fi_data.head()


Columns in fi_data:
['record_id', 'record_type', 'category', 'pillar', 'indicator', 'indicator_code', 'indicator_direction', 'value_numeric', 'value_text', 'value_type', 'unit', 'observation_date', 'period_start', 'period_end', 'fiscal_year', 'gender', 'location', 'region', 'source_name', 'source_type', 'source_url', 'confidence', 'related_indicator', 'relationship_type', 'impact_direction', 'impact_magnitude', 'impact_estimate', 'lag_months', 'evidence_basis', 'comparable_country', 'collected_by', 'collection_date', 'original_text', 'notes']


,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,REC_0001,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,22.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Baseline year,NaN
1,REC_0002,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,35.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
2,REC_0003,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,46.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
3,REC_0004,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,56.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN
4,REC_0005,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,36.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN


Sheet 1 contains measured data (observation), real events (event), and official targets (target).
Sheet 2 (impact_links) contains modeled relationships linking events to indicators via parent_id.
parent_id in impact_links points to the record_id of an event in Sheet 1. Each link defines the affected indicator, pillar, direction, magnitude, and lag.
Pillars cannot be assigned to events directly because one event may affect multiple indicators and pillars. For example, Telebirr launch affects both ACCESS and USAGE.

In [24]:
# List unique fields in reference codes
reference_codes['field'].unique()

# Example: check valid pillars
valid_pillars = reference_codes[reference_codes['field'] == 'pillar']['code'].tolist()
valid_record_types = reference_codes[reference_codes['field'] == 'record_type']['code'].tolist()
valid_categories = reference_codes[reference_codes['field'] == 'category']['code'].tolist()

print("Valid pillars:", valid_pillars)
print("Valid record types:", valid_record_types)
print("Valid event categories:", valid_categories)


Valid pillars: ['ACCESS', 'USAGE', 'QUALITY', 'AFFORDABILITY', 'TRUST', 'DEPTH', 'GENDER']
Valid record types: ['observation', 'event', 'impact_link', 'target', 'baseline', 'forecast']
Valid event categories: ['product_launch', 'market_entry', 'market_exit', 'policy', 'regulation', 'infrastructure', 'partnership', 'milestone', 'economic', 'pricing']


In [25]:
# Count by record_type
record_type_counts = fi_data['record_type'].value_counts()
# Count by pillar
pillar_counts = fi_data['pillar'].value_counts(dropna=False)
# Count by source_type
source_counts = fi_data['source_type'].value_counts()
# Count by confidence
confidence_counts = fi_data['confidence'].value_counts()

print("Record type counts:\n", record_type_counts)
print("\nPillar counts:\n", pillar_counts)
print("\nSource type counts:\n", source_counts)
print("\nConfidence counts:\n", confidence_counts)


Record type counts:
 record_type
observation    30
event          10
target          3
Name: count, dtype: int64

Pillar counts:
 pillar
ACCESS           16
USAGE            11
NaN              10
GENDER            5
AFFORDABILITY     1
Name: count, dtype: int64

Source type counts:
 source_type
operator      15
survey        10
regulator      7
research       4
policy         3
calculated     2
news           2
Name: count, dtype: int64

Confidence counts:
 confidence
high      40
medium     3
Name: count, dtype: int64


In [26]:
obs = fi_data[fi_data['record_type'] == 'observation']
print("Earliest observation date:", obs['observation_date'].min())
print("Latest observation date:", obs['observation_date'].max())


Earliest observation date: 2014-12-31 00:00:00
Latest observation date: 2025-12-31 00:00:00


In [27]:
indicator_counts = obs['indicator_code'].value_counts()
print("Indicator coverage:\n", indicator_counts)


Indicator coverage:
 indicator_code
ACC_OWNERSHIP         6
ACC_FAYDA             3
ACC_MM_ACCOUNT        2
ACC_4G_COV            2
USG_P2P_COUNT         2
GEN_GAP_ACC           2
ACC_MOBILE_PEN        1
USG_ATM_COUNT         1
USG_ATM_VALUE         1
USG_CROSSOVER         1
USG_P2P_VALUE         1
USG_TELEBIRR_USERS    1
USG_TELEBIRR_VALUE    1
USG_MPESA_ACTIVE      1
USG_MPESA_USERS       1
USG_ACTIVE_RATE       1
AFF_DATA_INCOME       1
GEN_MM_SHARE          1
GEN_GAP_MOBILE        1
Name: count, dtype: int64


This table shows which indicators have the most observations and which are sparse. Sparse indicators may need enrichment or careful modeling.

In [28]:
events = fi_data[fi_data['record_type'] == 'event']
events_summary = events[['record_id', 'category', 'indicator', 'observation_date']].sort_values('observation_date')
events_summary.head(10)


,record_id,category,indicator,observation_date
33,EVT_0001,product_launch,Telebirr Launch,2021-05-17
41,EVT_0009,policy,NFIS-II Strategy Launch,2021-09-01
34,EVT_0002,market_entry,Safaricom Ethiopia Commercial Launch,2022-08-01
35,EVT_0003,product_launch,M-Pesa Ethiopia Launch,2023-08-01
36,EVT_0004,infrastructure,Fayda Digital ID Program Rollout,2024-01-01
37,EVT_0005,policy,Foreign Exchange Liberalization,2024-07-29
38,EVT_0006,milestone,P2P Transaction Count Surpasses ATM,2024-10-01
39,EVT_0007,partnership,M-Pesa EthSwitch Integration,2025-10-27
42,EVT_0010,pricing,Safaricom Ethiopia Price Increase,2025-12-15
40,EVT_0008,infrastructure,EthioPay Instant Payment System Launch,2025-12-18


This table lists major events, their categories, and dates. For example, Telebirr launch (product_launch) and M-Pesa entry (market_entry).

In [29]:
impact_links_summary = impact_links[['parent_id', 'pillar', 'related_indicator', 'impact_direction', 'impact_magnitude', 'lag_months']]
impact_links_summary.head(10)


,parent_id,pillar,related_indicator,impact_direction,impact_magnitude,lag_months
0,EVT_0001,ACCESS,ACC_OWNERSHIP,increase,high,12
1,EVT_0001,USAGE,USG_TELEBIRR_USERS,increase,high,3
2,EVT_0001,USAGE,USG_P2P_COUNT,increase,high,6
3,EVT_0002,ACCESS,ACC_4G_COV,increase,medium,12
4,EVT_0002,AFFORDABILITY,AFF_DATA_INCOME,decrease,medium,12
5,EVT_0003,USAGE,USG_MPESA_USERS,increase,high,3
6,EVT_0003,ACCESS,ACC_MM_ACCOUNT,increase,medium,6
7,EVT_0004,ACCESS,ACC_OWNERSHIP,increase,medium,24
8,EVT_0004,GENDER,GEN_GAP_ACC,decrease,medium,24
9,EVT_0005,AFFORDABILITY,AFF_DATA_INCOME,increase,high,3


Each row links a parent event to an affected indicator.
impact_direction shows if the effect is positive/negative.
impact_magnitude shows strength (low/medium/high).
lag_months shows how long until the effect appears.